In [ ]:
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    await piplite.install("jinja2")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta
import ipywidgets as widgets

%matplotlib widget
pd.options.display.max_columns = None
pd.options.display.max_rows = None

---

# 11 Bullish Swings

Click $>>$ button to run the entire script first

**Buy Signal**
> BUY 100 shares when Aroon up(14,0) crosses above Aroon down(14,0) and Close(0) lower than Prev N(High,-1,day) at hour candle interval using Candlestick chart chart. Enter trade between 00:00 to 23:59

**Sell Signal**
> SELL 100 shares at stop loss % of 5 or target profit % of 10 at hour candle interval using Candlestick chart chart.

Please feel free to change to any **NSE SYMBOL** to the below list, separated by a comma

In [ ]:
symbols = "FACT"

In [ ]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "2mo"
intervalOfData = "60m"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        # print(text)
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas
    print(apiSymbolIndivi)

## An MACD line

<img src="https://commodity.com/wp-content/uploads/technical-analysis/Arooncompare.gif" width="700" height="300"/>

### Buy signal
> BUY 10 shares when Aroon up(14,0) crosses above Aroon down(14,0) and Close(0) lower than Prev N(High,-1,day) at hour candle interval using Candlestick chart chart. Enter trade between 00:00 to 23:59


- if AroonUp crosses AroonDown
    - $AroonUp[0]-AroonDown[0] > 0$
    - $AroonUp[-1] - AroonDown[-1] < 0$
- if HourClose lower than PreviousDailyHigh
    - $HourClose[0] < PrevDailyHigh[-1]$
    
### Sell signal
> SELL 10 shares at stop loss % of 2.5 or target profit % of 5 at hour candle interval using Candlestick chart chart.

If either of the below conditions are met
- Stop Loss condition met if $stock.hour[0] < order.buy*0.975$
- Target price condition met if $stock.close[0] > order.buy*1.05$

In [ ]:
display(Markdown("# Set the variables to this strategy here"))
display(Markdown("> We will improve UI shortly"))
display(Markdown("If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly"))
display(Markdown(f'{symbol}'))
@widgets.interact(_aroonPeriod=widgets.IntSlider(min=0, max=25, step=1, value=14,continuous_update=False),\
                  _stopLoss=widgets.FloatSlider(min=0, max=20, step=0.1, value=2.5,continuous_update=False),\
                  _targetPrice=widgets.FloatSlider(min=0, max=20, step=0.1, value=5,continuous_update=False),\
                  _numberOfStocks=widgets.IntSlider(min=0, max=100, step=1, value=10,continuous_update=False),\
                  _initialCash=widgets.IntSlider(min=0, max=100000, step=100, value=50000,continuous_update=False),
                 )
def voilas(_aroonPeriod,_stopLoss,_targetPrice,_numberOfStocks,_initialCash):
    class CrossOverStrat(bt.Strategy):
        def __init__(self):
            # Keep a reference to the hourly dataseries
            self.hour = self.datas[0]
            # Keep a reference to the daily dataseries
            self.prevDate = self.datas[1]
            #Some indicators
            self.AroonUpDown = bt.indicators.AroonUpDown(self.hour,period=_aroonPeriod)
            # self.AroonDown = bt.indicators.AroonDown(self.hour,period=aroonPeriod)

            # To keep track of pending orders and buy price/commission
            self.order = None
            self.buyprice = None
            self.buycomm = None

        def log(self, txt, dt=None):
            ''' Logging function for this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                else:  # Sell
                    self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            self.order = None

        def notify_trade(self, trade):
            if not trade.isclosed:
                return

            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

        def next(self):
            # Simply log the closing price of the series from the reference
            # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.order:
                return
            # Check if we are in the market
            if not self.position:
                if self.AroonUpDown.aroonup[0] - self.AroonUpDown.aroondown[0] > 0:
                    if self.AroonUpDown.aroonup[-1] - self.AroonUpDown.aroondown[-1] < 0:
                        if self.hour.close[0] < self.prevDate.high[-1]:
                            self.order = self.buy(size=_numberOfStocks)
            if self.position:
                # Already in the market ... we might sell
                if self.hour.close >= (1+_targetPrice/100)*self.buyprice:
                    self.log(f'target price reached')
                    self.order = self.sell(size=_numberOfStocks)
                if self.hour.close <= (1-_stopLoss/100)*self.buyprice:
                    self.log(f'stop loss reached')
                    self.order = self.sell(size=_numberOfStocks)
    cerebro = bt.Cerebro()

    data = bt.feeds.PandasData(dataname=stockPandasTot[symbol+".NS"][::-1],timeframe=bt.TimeFrame.Minutes)
    cerebro.adddata(data)
    cerebro.resampledata(data, timeframe=bt.TimeFrame.Days,compression=1)

    cerebro.addstrategy(CrossOverStrat)

    cerebro.broker.setcash(_initialCash)

    cerebro.broker.setcommission(commission=0.0025)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot(style='candlestick')

---
### Understanding above plot

> Please increase the size of the plot to the entire width of the screen, before reading this section. You can do the same by dragging the triangular element shown at the bottom right corner of the plot

The plot above has quite a lines and legends on them.
The legend is available on the left side indicating various items.

We shall move from the bottom most element and move upwards.
#### Aroon
The Aroon Indicator (often referred to as Aroon Up Down) is a range bound, technical indicator that is actually a set of two separate measurements designed to measure how many periods have passed since price has recorded an n-period high or low low with “n” being a number of periods set at the trader’s discretion. For example a 14 Day Aroon-Up will take the number of days since price last recorded a 14 day high and then calculate a number between 0 and 100.[(wiki)](https://www.tradingview.com/support/solutions/43000501801-aroon/)
#### Volume
The volume is indicated by bars of either gray or red colour. The respective value is availabe on the left part of the plot along the y-axis. Exact values for volume cannot be extracted currently. A red bar indicates a decrease in stock price over the time period. A grey bar indicates an increase in stock price over the given time period.
#### Close price line
The black line represents the close price line of the stock over a period of 48months, dates are visible at bottom of graph extracted from x value.

 *If you would like a candle stick pattern please add `style="candlestick"` to the arguments of the arguments of the plot command*
 
The default candlestick patterns are available over the course of the day, these follow the typical format with grey representing a rise in prices and red representing a fall in prices. Exact price and date values of cursor position are available at the bottom. The x-value represents date, while the y-value represents the price at cursor location.
#### Triangles
The triangles represent positions taken up according to the strategy mentioned by code above. A buy position/ signal will be generated if the triangle is green and pointed upwards, while a sell position/ sell signal is generated with the red downward pointing triangle.
#### Trading dots
The next canvas involves a set of dots representing either profit or losses which have occured by following the strategy. Here too the magnitudes can be separated by the colour of the dot and the cursor value on hovering over dots. One can also approximate the values with the data on the scale located on the right.
#### Portfolio value
The final screen on the top indicates the portfolio value and cash value. Each are differentiated by the respective color.

In [ ]:
symbols_02 = "BPCL,PGHH,SHREECEM,HDFC,COLPAL,KOTAKBANK,TORNTPHARM,ASIANPAINT,PIDILITIND,NESTLEIND,BHARTIARTL,SOBHA,BHEL,SAIL,GAIL,SRF"

In [ ]:
symbols_03 = "LICI,COALINDIA,EASEMYTRIP,IEX,CGPOWER,MSUMI,BEPL,IRCTC,CHEMPLASTS,CHENNPETRO,MARICO,TTML,NMDC,NATIONALUM,ITC,FACT,GNFC,VEDL,LSIL,GRAVITA,GUJGASLTD,KPITTECH,RITES,UGARSUGAR,JSWSTEEL,MADRASFERT,FINPIPE,BEL,PETRONET,ASHOKA,MAZDOCK,ENGINERSIN,DATAMATICS,JAMNAAUTO,MOREPENLAB,MARKSANS,ZENTEC,SAIL,CROMPTON,DABUR,CHAMBLFERT,TRIDENT,GAIL,SPIC,NBCC,SONACOMS,BLS,LAURUSLABS,TRITURBINE,MIRZAINT"

In [ ]:
symbols_04 = "IRFC,RVNL,ZOMATO,MANAPPURAM,IRCON,BHEL,ENGINERSIN,PNB,IDFCFIRSTB,TATASTEEL,PATELENG,FEDERALBNK,MAHABANK,BANKBARODA,HUDCO,BANKINDIA,ICICIBANK,J&KBANK,UCOBANK,SJVN,NBCC,SAIL,CENTRALBK,UNIONBANK,SBIN,NHPC,IOB,NYKAA,BEL,AXISBANK,HSCL,TATAMOTORS,M&MFIN,CANBK,RECLTD,UJJIVANSFB,IOC,JISLJALEQS,ADANIPOWER,TATAPOWER,RENUKA,IDFC,L&TFH,IBULHSGFIN,AMBUJACEM,EQUITASBNK,GSFC,MOTHERSON,PFC,GMRINFRA,ITC,ONGC,IRB,MMTC,MSUMI,INDHOTEL,RBLBANK,IEX,NATIONALUM,GICRE,NMDC,LEMONTREE,ABFRL,TRIDENT,NIACL,ITI,NTPC,HFCL,PSB,PAYTM,JKTYRE,NFL,RAILTEL,LSIL,GAIL"

In [ ]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "2mo"
intervalOfData = "60m"
apiSymbol = []
for symbol in symbols_03.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot_02 = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot_02[apiSymbolIndivi] = stockPandas

In [ ]:
## Variables
# Number of days checked [Can we push this dates checked to different cell?]
datesChecked = 15
# weights of each individual day (Shape in reshape function should be matched. )
weights = np.array([1, 15, 10, 5, 2])
# Define the strategy values here
aroonPeriod = 14
# Adding additional weights, will be helfull only while checking.
weights = np.append(weights, np.ones((datesChecked-5,1)))
signalBullDict = {}

In [ ]:
for symbol_02 in stockPandasTot_02:
    # Reversing data because finta likes calculations with oldest date on top
    selectData = stockPandasTot_02[symbol_02][::-1]
    # These data are reversed again to show the latest date at the top row.
    # Calculate the high-low periods
    # This is how the Aroon signature is calculated
    high_period = selectData['high'].rolling(aroonPeriod + 1).apply(lambda x: x.argmax())
    low_period = selectData['low'].rolling(aroonPeriod + 1).apply(lambda x: x.argmin())
    # Calculate the Aroon Up and Aroon Down
    aroon_up =100 - (((aroonPeriod - high_period) / aroonPeriod) * 100)
    # the aroon calculations are mentioned in the wiki mentioned below
    aroon_up = aroon_up[::-1]
    aroon_down =100 - (((aroonPeriod - low_period) / aroonPeriod) * 100)
    aroon_down = aroon_down[::-1]
    diff_0 = (aroon_up - aroon_down).fillna(0)
    diff_1 = (aroon_up.shift(-1) - aroon_down.shift(-1)).fillna(0)
    signChangeGood = ((diff_0 > 0) & (diff_1 < 0)).astype(int).to_numpy()
    signChange = signChangeGood
    signChange = signChange[0:datesChecked]
    signalBull = weights*signChange
    signalBullDict[symbol_02] = signalBull

In [ ]:
signalBullPand = pd.DataFrame(signalBullDict).T

def color_mapper(row):
    # compute the sum of the row
    sum = row.sum()
    # if the sum is negative, return red
    if sum < 0:
        return ['color: white; background-color: rgb(120, 50, 20)'] * len(row)
    # if the sum is positive, map it to a shade of green based on its magnitude
    else:
        magnitude = sum*50
        r = 15
        g = magnitude
        b = 15
        return [f'color: white; background-color: rgb({r},{g},{b})'] * len(row)

def make_pretty(styler):
    styler.apply(color_mapper,axis=1)
    styler.format(precision=1)
    return styler

signalBullPand.style.pipe(make_pretty)